# Sudoku Project
### December 2017, by Jude Moon

windows, Python3.6, [aind-environment-windows.yml](http://video.udacity-data.com.s3.amazonaws.com/topher/2017/April/58ee7f6c_aind-environment-windows/aind-environment-windows.yml)
***

## What is Sudoku?
It consists of a 9x9 grid, and the objective is to fill the grid with digits in such a way that each row, each column, and each of the 9 principal 3x3 subsquares contains all of the digits from 1 to 9. 
![image](https://d17h27t6h515a5.cloudfront.net/topher/2017/January/586c7a2a_labels/labels.png)

## Terminology for Sudoku

- **Box**: The individual squares at the intersection of rows and columns will be called boxes
- **Unit**: The complete rows, columns, and 3x3 squares, will be called units
- **Peer**: For a particular box (such as 'A1'), its peers will be all other boxes that belong to a common unit (namely, those that belong to the same row, column, or 3x3 square).

![image](https://d17h27t6h515a5.cloudfront.net/topher/2017/January/5872f924_peers/peers.png)

## Project Goals
- Additional **diagonal rule**: A diagonal sudoku is like a regular sudoku, except that among the two main diagonals, the numbers 1 to 9 should all appear exactly once.
- Sudoku solution using **search and constraint propagation**
- Implementation of the **naked twins technique**. 

In [1]:
# all the possible concatenations of a letter 's' in string 'a' with a letter 't' in string 'b'
def cross(a, b):
    return [s+t for s in a for t in b]

In [2]:
# define units and peers for 9x9 grid
rows = 'ABCDEFGHI'
cols = '123456789'
boxes = cross(rows, cols)
row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
# Additional diagonal units
diagonal_units = [[rows[i] + cols[i] for i in range(len(rows))],
                  [rows[-(i+1)] + cols[i] for i in range(len(rows))]]
unitlist = row_units + column_units + square_units + diagonal_units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

In [3]:
def str2dict(string):
    """
    Input: Sudoku grid in string form, 81 characters long
    Procedure: Convert grid string into {<box>: <value>} dict with '123456789' value for empties   
    Output: Sudoku grid in dictionary form
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    assert len(string) == 81, "Input grid must be a string of length 81 (9x9)"
    values = []
    for c in string:
        if c == '.':
            values.append('123456789')
        else:
            values.append(c)
    return dict(zip(boxes, values))

In [4]:
def display(dictionary):
    """
    Input: The sudoku in dictionary form
    Procedure: Display the dictionary as a 2-D grid
    """
    width = 1+max(len(dictionary[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(dictionary[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)

In [5]:
string_sudoku = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
display(str2dict(string_sudoku))

    2     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789     6     |    2     123456789 123456789 
123456789 123456789     1     |123456789 123456789 123456789 |123456789     7     123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     6     |123456789 123456789     8     |123456789 123456789 123456789 
    3     123456789 123456789 |123456789     9     123456789 |123456789 123456789     7     
123456789 123456789 123456789 |    6     123456789 123456789 |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789     4     123456789 |123456789 123456789 123456789 |    8     123456789 123456789 
123456789 123456789     5     |    2     123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789 123456789 |12345678

## Strategy 1: Elimination
Replace '.' with all the available values for that box

In [6]:
def eliminate(dictionary):
    """
    Input: Sudoku in dictionary form
    Procedure: Go through all the boxes, and whenever there is a box with a single value,
               eliminate this value from the set of values of all its peers   
    Output: Resulting Sudoku in dictionary form after eliminating values
    """
    solved_values = [box for box in dictionary.keys() if len(dictionary[box]) == 1]
    for box in solved_values:
        digit = dictionary[box]
        for peer in peers[box]:
            dictionary[peer] = dictionary[peer].replace(digit,'')
    return dictionary

In [7]:
display(eliminate(str2dict(string_sudoku)))

   2     356789  34789  |1345789  134578  134579 | 13569  1345689   145   
 45789     57    34789  |1345789  134578    6    |   2      1345   14589  
 45689   35689     1    | 34589   23458   23459  |   35      7     45689  
------------------------+------------------------+------------------------
 14579   12579     6    |  457    123457    8    |  1359   12359    1259  
   3      1258    248   |  145      9      1245  |  156    12568     7    
 15789   125789   2789  |   6     12357     57   |   4     123589  12589  
------------------------+------------------------+------------------------
  1679     4      237   | 13579   13567   13579  |   8     12569   12569  
 16789    137      5    |   2     134678  13479  |  1679     46     1469  
   17    126789   2789  | 145789  145678  14579  | 15679   124569    3    


## Strategy 2: Only Choice
If there is only one box in a unit which would allow a certain digit, then that box must be assigned that digit.

In [8]:
def only_choice(dictionary):
    """
    Input: Sudoku in dictionary form
    Procedure: Go through all the units, and whenever there is a unit with a value
               that only fits in one box, assign the value to this box  
    Output: Resulting Sudoku in dictionary form filling in only choices
    """
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in dictionary[box]]
            if len(dplaces) == 1:
                dictionary[dplaces[0]] = digit
    return dictionary

In [9]:
display(only_choice(eliminate(str2dict(string_sudoku))))

   2     356789  34789  |1345789  134578  134579 | 13569  1345689   145   
 45789     57    34789  |1345789  134578    6    |   2      1345   14589  
 45689   35689     1    | 34589   23458   23459  |   35      7     45689  
------------------------+------------------------+------------------------
 14579   12579     6    |  457    123457    8    |  1359   12359    1259  
   3      1258    248   |  145      9      1245  |  156    12568     7    
 15789   125789   2789  |   6     12357     57   |   4     123589  12589  
------------------------+------------------------+------------------------
  1679     4       2    | 13579   13567   13579  |   8     12569   12569  
 16789    137      5    |   2     134678  13479  |  1679     6      1469  
   17    126789   2789  | 145789  145678  14579  | 15679   124569    3    


In [10]:
def iterate(dictionary):
    """
    Input: Sudoku in dictionary form
    Procedure:  Iterate eliminate() and only_choice(). 
                If at some point, there is a box with no available values, return False.
                If the sudoku is solved, return the sudoku.
                If after an iteration of both functions, the sudoku remains the same, return the sudoku
    Output: Solved Sudoku in dictionary form 
    """
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in dictionary.keys() if len(dictionary[box]) == 1])
        # Use the Eliminate Strategy
        dictionary = eliminate(dictionary)
        # Use the Only Choice Strategy
        dictionary = only_choice(dictionary)
        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in dictionary.keys() if len(dictionary[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in dictionary.keys() if len(dictionary[box]) == 0]):
            return False
    return dictionary

In [11]:
if iterate(str2dict(string_sudoku)) != False:
    display(iterate(str2dict(string_sudoku)))
else:
    print(False)

2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 


## Strategy 3: Depth First Search
- Construct tree of possibility
- Fist, pick a box with a minimal number of possible values.
- If we don't solve the Sudoku and we get a few possibilies for some boxes will branch out and try to solve the sudoku's obtain its way. 
- Go through the possibilities untill we reach a solution.
source: [AlphaGo](https://storage.googleapis.com/deepmind-media/alphago/AlphaGoNaturePaper.pdf)

In [12]:
def search(dictionary):
    """
    Input: Sudoku in dictionary form
    Procedure: Depth First Search
    Output: Solved Sudoku in dictionary form 
    """
    
    # First, Iterate eliminate() and only_choice() using the previous procedure
    dictionary = iterate(dictionary)
    if dictionary is False:
        return False ## Failed earlier
    if all(len(dictionary[s]) == 1 for s in boxes): 
        return dictionary ## Solved!
    
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(dictionary[s]), s) for s in boxes if len(dictionary[s]) > 1)
    
    # Now use recurrence to solve each one of the resulting sudokus, 
    # and if one returns a value (not False), return that answer!
    for value in dictionary[s]:
        new_sudoku = dictionary.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt

In [13]:
def solve(string):
    """
    Input: Sudoku in string form
    Procedure: Find the solution to a Sudoku puzzle using search and constraint propagation
    Output: Solved Sudoku in dictionary form 
    """
    if iterate(str2dict(string_sudoku)) == False:
        return False
    else:
        return search(str2dict(string))

In [14]:
display(solve(string_sudoku))

2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 


### Test diagonal units

In [15]:
diagonal_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
solved_diag_sudoku = {'G7': '8', 'G6': '9', 'G5': '7', 'G4': '3', 'G3': '2', 'G2': '4', 'G1': '6', 'G9': '5',
                          'G8': '1', 'C9': '6', 'C8': '7', 'C3': '1', 'C2': '9', 'C1': '4', 'C7': '5', 'C6': '3',
                          'C5': '2', 'C4': '8', 'E5': '9', 'E4': '1', 'F1': '1', 'F2': '2', 'F3': '9', 'F4': '6',
                          'F5': '5', 'F6': '7', 'F7': '4', 'F8': '3', 'F9': '8', 'B4': '7', 'B5': '1', 'B6': '6',
                          'B7': '2', 'B1': '8', 'B2': '5', 'B3': '3', 'B8': '4', 'B9': '9', 'I9': '3', 'I8': '2',
                          'I1': '7', 'I3': '8', 'I2': '1', 'I5': '6', 'I4': '5', 'I7': '9', 'I6': '4', 'A1': '2',
                          'A3': '7', 'A2': '6', 'E9': '7', 'A4': '9', 'A7': '3', 'A6': '5', 'A9': '1', 'A8': '8',
                          'E7': '6', 'E6': '2', 'E1': '3', 'E3': '4', 'E2': '8', 'E8': '5', 'A5': '4', 'H8': '6',
                          'H9': '4', 'H2': '3', 'H3': '5', 'H1': '9', 'H6': '1', 'H7': '7', 'H4': '2', 'H5': '8',
                          'D8': '9', 'D9': '2', 'D6': '8', 'D7': '1', 'D4': '4', 'D5': '3', 'D2': '7', 'D3': '6',
                          'D1': '5'}

In [16]:
my_solution = solve(diagonal_grid)
if my_solution == solved_diag_sudoku:
    print(True)
else:
    print(False)

True


## Implementation of [Naked Twins Technique](http://www.sudokudragon.com/tutorialnakedtwins.htm)
1. Find twins which include the same possible values within a unit.
2. Lock the values in the twin boxes only.
3. Remove the locked values from other boxes in the same unit.

In [17]:
def naked_twins(dictionary):
    """
    Input: Sudoku in dictionary form
    Procedure: Naked Twins Technique
    Output: Solved Sudoku in dictionary form
    """
    # find all boxes with two values
    for unit in unitlist:
        pairs = [box for box in unit if len(dictionary[box]) == 2]
        if len(pairs) > 1:
            twin_dict = {} # pair_value as a key, list of pair as a value
            for pair in pairs:
                pair_value = dictionary[pair]
                if pair_value in twin_dict:
                    twin_dict[pair_value].append(pair)
                else:
                    twin_dict[pair_value] = [pair]
            # remove key with the number of pairs != 2
            naked_twins = {k: v for k, v in twin_dict.items() if len(v) == 2}
            # if there is naked twins, get all values of naked twins
            if naked_twins:
                naked_twins_values = ''
                for v in naked_twins:
                    naked_twins_values = naked_twins_values + v
                # eliminate the values of the naked twins from non-twins values
                for box in unit:
                    if len(dictionary[box])>2:
                        for digit in dictionary[box]:
                            if digit in naked_twins_values:
                                dictionary[box] = dictionary[box].replace(digit,'')
    return dictionary


### Test naked_twins

In [18]:
def test_naked_twins(problem, possible_solutions):
    print('Before naked_twins:')
    display(problem)
    
    my_solution = naked_twins(problem)
    print('After naked_twins:')
    display(my_solution)
    print('Match with solution?')
    if my_solution in possible_solutions:
        print(True)
    else:
        print(False)

In [19]:
# problem 1
problem_1 = {'I6': '4', 'H9': '3', 'I2': '6', 'E8': '1', 'H3': '5', 'H7': '8', 'I7': '1', 'I4': '8',
                            'H5': '6', 'F9': '7', 'G7': '6', 'G6': '3', 'G5': '2', 'E1': '8', 'G3': '1', 'G2': '8',
                            'G1': '7', 'I1': '23', 'C8': '5', 'I3': '23', 'E5': '347', 'I5': '5', 'C9': '1', 'G9': '5',
                            'G8': '4', 'A1': '1', 'A3': '4', 'A2': '237', 'A5': '9', 'A4': '2357', 'A7': '27',
                            'A6': '257', 'C3': '8', 'C2': '237', 'C1': '23', 'E6': '579', 'C7': '9', 'C6': '6',
                            'C5': '37', 'C4': '4', 'I9': '9', 'D8': '8', 'I8': '7', 'E4': '6', 'D9': '6', 'H8': '2',
                            'F6': '125', 'A9': '8', 'G4': '9', 'A8': '6', 'E7': '345', 'E3': '379', 'F1': '6',
                            'F2': '4', 'F3': '23', 'F4': '1235', 'F5': '8', 'E2': '37', 'F7': '35', 'F8': '9',
                            'D2': '1', 'H1': '4', 'H6': '17', 'H2': '9', 'H4': '17', 'D3': '2379', 'B4': '27',
                            'B5': '1', 'B6': '8', 'B7': '27', 'E9': '2', 'B1': '9', 'B2': '5', 'B3': '6', 'D6': '279',
                            'D7': '34', 'D4': '237', 'D5': '347', 'B8': '3', 'B9': '4', 'D1': '5'}

# possible solution 1
possible_solutions_1 = [
        {'G7': '6', 'G6': '3', 'G5': '2', 'G4': '9', 'G3': '1', 'G2': '8', 'G1': '7', 'G9': '5', 'G8': '4', 'C9': '1',
         'C8': '5', 'C3': '8', 'C2': '237', 'C1': '23', 'C7': '9', 'C6': '6', 'C5': '37', 'A4': '2357', 'A9': '8',
         'A8': '6', 'F1': '6', 'F2': '4', 'F3': '23', 'F4': '1235', 'F5': '8', 'F6': '125', 'F7': '35', 'F8': '9',
         'F9': '7', 'B4': '27', 'B5': '1', 'B6': '8', 'B7': '27', 'E9': '2', 'B1': '9', 'B2': '5', 'B3': '6', 'C4': '4',
         'B8': '3', 'B9': '4', 'I9': '9', 'I8': '7', 'I1': '23', 'I3': '23', 'I2': '6', 'I5': '5', 'I4': '8', 'I7': '1',
         'I6': '4', 'A1': '1', 'A3': '4', 'A2': '237', 'A5': '9', 'E8': '1', 'A7': '27', 'A6': '257', 'E5': '347',
         'E4': '6', 'E7': '345', 'E6': '579', 'E1': '8', 'E3': '79', 'E2': '37', 'H8': '2', 'H9': '3', 'H2': '9',
         'H3': '5', 'H1': '4', 'H6': '17', 'H7': '8', 'H4': '17', 'H5': '6', 'D8': '8', 'D9': '6', 'D6': '279',
         'D7': '34', 'D4': '237', 'D5': '347', 'D2': '1', 'D3': '79', 'D1': '5'},
        {'I6': '4', 'H9': '3', 'I2': '6', 'E8': '1', 'H3': '5', 'H7': '8', 'I7': '1', 'I4': '8', 'H5': '6', 'F9': '7',
         'G7': '6', 'G6': '3', 'G5': '2', 'E1': '8', 'G3': '1', 'G2': '8', 'G1': '7', 'I1': '23', 'C8': '5', 'I3': '23',
         'E5': '347', 'I5': '5', 'C9': '1', 'G9': '5', 'G8': '4', 'A1': '1', 'A3': '4', 'A2': '237', 'A5': '9',
         'A4': '2357', 'A7': '27', 'A6': '257', 'C3': '8', 'C2': '237', 'C1': '23', 'E6': '579', 'C7': '9', 'C6': '6',
         'C5': '37', 'C4': '4', 'I9': '9', 'D8': '8', 'I8': '7', 'E4': '6', 'D9': '6', 'H8': '2', 'F6': '125',
         'A9': '8', 'G4': '9', 'A8': '6', 'E7': '345', 'E3': '79', 'F1': '6', 'F2': '4', 'F3': '23', 'F4': '1235',
         'F5': '8', 'E2': '3', 'F7': '35', 'F8': '9', 'D2': '1', 'H1': '4', 'H6': '17', 'H2': '9', 'H4': '17',
         'D3': '79', 'B4': '27', 'B5': '1', 'B6': '8', 'B7': '27', 'E9': '2', 'B1': '9', 'B2': '5', 'B3': '6',
         'D6': '279', 'D7': '34', 'D4': '237', 'D5': '347', 'B8': '3', 'B9': '4', 'D1': '5'}]

test_naked_twins(problem_1, possible_solutions_1)

Before naked_twins:
  1   237   4  | 2357  9   257 |  27   6    8  
  9    5    6  |  27   1    8  |  27   3    4  
  23  237   8  |  4    37   6  |  9    5    1  
---------------+---------------+---------------
  5    1   2379| 237  347  279 |  34   8    6  
  8    37  379 |  6   347  579 | 345   1    2  
  6    4    23 | 1235  8   125 |  35   9    7  
---------------+---------------+---------------
  7    8    1  |  9    2    3  |  6    4    5  
  4    9    5  |  17   6    17 |  8    2    3  
  23   6    23 |  8    5    4  |  1    7    9  
After naked_twins:
  1   237   4  | 2357  9   257 |  27   6    8  
  9    5    6  |  27   1    8  |  27   3    4  
  23  237   8  |  4    37   6  |  9    5    1  
---------------+---------------+---------------
  5    1    79 | 237  347  279 |  34   8    6  
  8    37   79 |  6   347  579 | 345   1    2  
  6    4    23 | 1235  8   125 |  35   9    7  
---------------+---------------+---------------
  7    8    1  |  9    2    3  |  6    4    5  
 

In [20]:
# problem 2
problem_2 = {'A1': '23', 'A2': '4', 'A3': '7', 'A4': '6', 'A5': '8', 'A6': '5', 'A7': '23', 'A8': '9',
                            'A9': '1', 'B1': '6', 'B2': '9', 'B3': '8', 'B4': '4', 'B5': '37', 'B6': '1', 'B7': '237',
                            'B8': '5', 'B9': '237', 'C1': '23', 'C2': '5', 'C3': '1', 'C4': '23', 'C5': '379',
                            'C6': '2379', 'C7': '8', 'C8': '6', 'C9': '4', 'D1': '8', 'D2': '17', 'D3': '9',
                            'D4': '1235', 'D5': '6', 'D6': '237', 'D7': '4', 'D8': '27', 'D9': '2357', 'E1': '5',
                            'E2': '6', 'E3': '2', 'E4': '8', 'E5': '347', 'E6': '347', 'E7': '37', 'E8': '1', 'E9': '9',
                            'F1': '4', 'F2': '17', 'F3': '3', 'F4': '125', 'F5': '579', 'F6': '279', 'F7': '6',
                            'F8': '8', 'F9': '257', 'G1': '1', 'G2': '8', 'G3': '6', 'G4': '35', 'G5': '345',
                            'G6': '34', 'G7': '9', 'G8': '27', 'G9': '27', 'H1': '7', 'H2': '2', 'H3': '4', 'H4': '9',
                            'H5': '1', 'H6': '8', 'H7': '5', 'H8': '3', 'H9': '6', 'I1': '9', 'I2': '3', 'I3': '5',
                            'I4': '7', 'I5': '2', 'I6': '6', 'I7': '1', 'I8': '4', 'I9': '8'}

# possible solution 2
possible_solutions_2 = [
        {'A1': '23', 'A2': '4', 'A3': '7', 'A4': '6', 'A5': '8', 'A6': '5', 'A7': '23', 'A8': '9', 'A9': '1', 'B1': '6',
         'B2': '9', 'B3': '8', 'B4': '4', 'B5': '37', 'B6': '1', 'B7': '237', 'B8': '5', 'B9': '237', 'C1': '23',
         'C2': '5', 'C3': '1', 'C4': '23', 'C5': '79', 'C6': '79', 'C7': '8', 'C8': '6', 'C9': '4', 'D1': '8',
         'D2': '17', 'D3': '9', 'D4': '1235', 'D5': '6', 'D6': '237', 'D7': '4', 'D8': '27', 'D9': '2357', 'E1': '5',
         'E2': '6', 'E3': '2', 'E4': '8', 'E5': '347', 'E6': '347', 'E7': '37', 'E8': '1', 'E9': '9', 'F1': '4',
         'F2': '17', 'F3': '3', 'F4': '125', 'F5': '579', 'F6': '279', 'F7': '6', 'F8': '8', 'F9': '257', 'G1': '1',
         'G2': '8', 'G3': '6', 'G4': '35', 'G5': '345', 'G6': '34', 'G7': '9', 'G8': '27', 'G9': '27', 'H1': '7',
         'H2': '2', 'H3': '4', 'H4': '9', 'H5': '1', 'H6': '8', 'H7': '5', 'H8': '3', 'H9': '6', 'I1': '9', 'I2': '3',
         'I3': '5', 'I4': '7', 'I5': '2', 'I6': '6', 'I7': '1', 'I8': '4', 'I9': '8'},
        {'A1': '23', 'A2': '4', 'A3': '7', 'A4': '6', 'A5': '8', 'A6': '5', 'A7': '23', 'A8': '9', 'A9': '1', 'B1': '6',
         'B2': '9', 'B3': '8', 'B4': '4', 'B5': '3', 'B6': '1', 'B7': '237', 'B8': '5', 'B9': '237', 'C1': '23',
         'C2': '5', 'C3': '1', 'C4': '23', 'C5': '79', 'C6': '79', 'C7': '8', 'C8': '6', 'C9': '4', 'D1': '8',
         'D2': '17', 'D3': '9', 'D4': '1235', 'D5': '6', 'D6': '237', 'D7': '4', 'D8': '27', 'D9': '2357', 'E1': '5',
         'E2': '6', 'E3': '2', 'E4': '8', 'E5': '347', 'E6': '347', 'E7': '37', 'E8': '1', 'E9': '9', 'F1': '4',
         'F2': '17', 'F3': '3', 'F4': '125', 'F5': '579', 'F6': '279', 'F7': '6', 'F8': '8', 'F9': '257', 'G1': '1',
         'G2': '8', 'G3': '6', 'G4': '35', 'G5': '345', 'G6': '34', 'G7': '9', 'G8': '27', 'G9': '27', 'H1': '7',
         'H2': '2', 'H3': '4', 'H4': '9', 'H5': '1', 'H6': '8', 'H7': '5', 'H8': '3', 'H9': '6', 'I1': '9', 'I2': '3',
         'I3': '5', 'I4': '7', 'I5': '2', 'I6': '6', 'I7': '1', 'I8': '4', 'I9': '8'}]

test_naked_twins(problem_2, possible_solutions_2)

Before naked_twins:
  23   4    7  |  6    8    5  |  23   9    1  
  6    9    8  |  4    37   1  | 237   5   237 
  23   5    1  |  23  379  2379|  8    6    4  
---------------+---------------+---------------
  8    17   9  | 1235  6   237 |  4    27  2357
  5    6    2  |  8   347  347 |  37   1    9  
  4    17   3  | 125  579  279 |  6    8   257 
---------------+---------------+---------------
  1    8    6  |  35  345   34 |  9    27   27 
  7    2    4  |  9    1    8  |  5    3    6  
  9    3    5  |  7    2    6  |  1    4    8  
After naked_twins:
  23   4    7  |  6    8    5  |  23   9    1  
  6    9    8  |  4    37   1  | 237   5   237 
  23   5    1  |  23   79   79 |  8    6    4  
---------------+---------------+---------------
  8    17   9  | 1235  6   237 |  4    27  2357
  5    6    2  |  8   347  347 |  37   1    9  
  4    17   3  | 125  579  279 |  6    8   257 
---------------+---------------+---------------
  1    8    6  |  35  345   34 |  9    27   27 
 